In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
da = pd.read_csv('IMDB_Review.csv', on_bad_lines='skip',encoding='utf-8')

In [4]:
import nltk
from nltk.corpus import stopwords

In [59]:
da = da.sample(n=2000, random_state=42)

In [42]:
da = da.apply(lambda x : x.str.lower() if x.dtype == 'object' else x)

In [43]:
import re
url_pattern = re.compile(r'https?://\S+|www\.\S+')
da['review'] = da['review'].apply(lambda x: url_pattern.sub('',x))

In [44]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [45]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoding = tokenizer(
    list(da['review']),
    truncation=True,
    padding='max_length',
    max_length= 128,
    return_tensors='pt'
)

In [46]:
da['label'] = da['sentiment'].map({'positive' : 1 , 'negative' : 0})

In [47]:
import torch
from torch.utils.data import TensorDataset
labels = torch.tensor(da['label'].values)
dataset = TensorDataset(encoding['input_ids'],encoding['attention_mask'],labels)

In [48]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)

In [49]:
dataset = SentimentDataset(encoding,labels)

In [50]:
from torch.utils.data import random_split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset,[train_size, val_size])

In [51]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16)

In [61]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [62]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)
    return {'accuracy': accuracy_score(labels, predictions)}

In [63]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

<ipython-input-48-fb1fe15b830c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(self.labels[idx], dtype=torch.long)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.470000,0.488162,0.817500
2,0.212400,0.665251,0.835000


<ipython-input-48-fb1fe15b830c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(self.labels[idx], dtype=torch.long)


TrainOutput(global_step=400, training_loss=0.28022936955094335, metrics={'train_runtime': 3081.4264, 'train_samples_per_second': 1.038, 'train_steps_per_second': 0.13, 'total_flos': 210488844288000.0, 'train_loss': 0.28022936955094335, 'epoch': 2.0})

In [57]:
trainer.evaluate()

<ipython-input-48-fb1fe15b830c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(self.labels[idx], dtype=torch.long)


{'eval_loss': 0.3697516620159149,
 'eval_accuracy': 0.86,
 'eval_runtime': 112.109,
 'eval_samples_per_second': 3.568,
 'eval_steps_per_second': 0.446,
 'epoch': 1.0}